In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
import tensorflow as tf
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, activations, models, preprocessing, utils
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from nltk.tokenize import word_tokenize, sent_tokenize
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import nltk
import sqlite3
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
with open('/content/classification_v2.json', 'r') as j:
     contents = json.loads(j.read())

def preprocess_and_tokenize_sentences(text):
    text = text.lower()
    # text = re.sub(r'[^a-zA-Z\s?.]', '', text)
    text = text.replace('?','')
    sentences = sent_tokenize(text)
    return sentences

In [3]:
chatbot_data= pd.read_csv('/content/chatbot_dataset_4.csv')
chatbot_data['cauhoi'] = chatbot_data['cauhoi'].astype(str)
chatbot_data['traloi'] = chatbot_data['traloi'].astype(str)
questions= chatbot_data['cauhoi'].tolist()

In [4]:
tags_list=[]
sentences_list= []
for intents in contents['Intenst']:
    tags= intents['Tags']
    questions = intents['Questions']
    for question in questions:
        tags_list.append(tags)
        sentences = preprocess_and_tokenize_sentences(question)
        # print(sentences)
        sentences_list.extend(sentences)

In [5]:
classification_data = pd.DataFrame({'Sentence': sentences_list,'Tag': tags_list})
classification_data = classification_data.sample(frac=1, random_state=42).reset_index(drop=True)
classification_data.to_csv('classification.csv',index=False)

In [6]:
classification_data = pd.read_csv('/content/classification.csv')
classification_data.head()

,Sentence,Tag
0,what financial outlay is expected for the acqu...,Information tags
1,what would be the purchase cost including taxe...,Information tags
2,what would the xiaomi redmi note 11 4gb-128gb ...,Information tags
3,am i able to reserve items that aren't in stoc...,Non-Informations tags
4,could i see a demonstration of the product bef...,Non-Informations tags


In [7]:
labels_mapping = {
    'Information tags': 1,
    'Non-Informations tags': 0
}
classification_data['Tag'] = classification_data['Tag'].map(labels_mapping)

In [8]:
X = classification_data.Sentence
Y = classification_data.Tag
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
classification_data.sample(10)

all_texts = classification_data['Sentence'].tolist() + chatbot_data['cauhoi'].tolist() + chatbot_data['traloi'].tolist()

In [9]:
max_len = 18
tokenizer = Tokenizer(oov_token='OOV')
tokenizer.fit_on_texts(all_texts)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
# print(X_train_sequences)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')

In [10]:
def model():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(vocab_size,30,input_length=max_len)(inputs)
    layer = LSTM(16)(layer)
    layer = Dense(8,name='FC1')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model_1 = model()
model_1.summary()
model_1.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 18)]              0         
                                                                 
 embedding (Embedding)       (None, 18, 30)            86160     
                                                                 
 lstm (LSTM)                 (None, 16)                3008      
                                                                 
 FC1 (Dense)                 (None, 8)                 136       
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 out_layer (Dense)           (None, 1)                 9         
                                                                 
 activation (Activation)     (None, 1)                 0     

In [11]:
history=model_1.fit(X_train_padded, Y_train, validation_data=(X_test_padded, Y_test) ,epochs=10,batch_size=2)

Epoch 1/10
90/90 [==============================] - 17s 98ms/step - loss: 0.6341 - accuracy: 0.6722 - val_loss: 0.3564 - val_accuracy: 0.9524
Epoch 2/10
90/90 [==============================] - 1s 7ms/step - loss: 0.1312 - accuracy: 0.9722 - val_loss: 0.1858 - val_accuracy: 0.9524
Epoch 3/10
90/90 [==============================] - 1s 8ms/step - loss: 0.0288 - accuracy: 1.0000 - val_loss: 0.0209 - val_accuracy: 1.0000
Epoch 4/10
90/90 [==============================] - 1s 6ms/step - loss: 0.0229 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 5/10
90/90 [==============================] - 1s 6ms/step - loss: 0.0130 - accuracy: 0.9944 - val_loss: 4.9691e-04 - val_accuracy: 1.0000
Epoch 6/10
90/90 [==============================] - 1s 6ms/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 1.0910e-04 - val_accuracy: 1.0000
Epoch 7/10
90/90 [==============================] - 1s 6ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 5.3360e-05 - val_accuracy: 1.0000
Epoch 

In [12]:
questions = chatbot_data['cauhoi'].tolist()
answers = chatbot_data["traloi"].to_list()
answers_with_tags = list()
#lấy ra từ điển
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )
answers = list()

#gan nhan
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

# tokenizer = preprocessing.text.Tokenizer()
# tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index ) + 1
# print( 'Số lượng từ trong từ điển: {}'.format( VOCAB_SIZE ))

#trích thành các từ xl các kí tự đặc biệt
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

In [13]:
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)

tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
# print(  )
# print("Decoder_input_data",decoder_input_data.shape , maxlen_answers)
# print(answers[0])
# print(decoder_input_data[0])

# decoder_output_data giải mã từ vector số sang string
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
# print(  )
# print("Decoder_output_data", decoder_output_data.shape)
# print(answers[0])
# print(decoder_output_data[0])

#khởi tạo mô hình lstm
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]
print(encoder_outputs)

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )
print(output)

model_2= tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model_2.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model_2.summary()


KerasTensor(type_spec=TensorSpec(shape=(None, 200), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:0', description="created by layer 'lstm_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 110, 2872), dtype=tf.float32, name=None), name='dense/Softmax:0', description="created by layer 'dense'")
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 40)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 110)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 40, 200)              574400    ['input_1[0][

In [14]:
model_2.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size= 1, epochs=3 )

Epoch 1/3
857/857 [==============================] - 28s 19ms/step - loss: 6.1052
Epoch 2/3
857/857 [==============================] - 10s 12ms/step - loss: 5.7854
Epoch 3/3
857/857 [==============================] - 10s 11ms/step - loss: 5.6347


In [15]:
def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    print(encoder_model)
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)

    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

def str_to_tokens( sentence : str ):

    words = sentence.lower().replace("?","").split()
    tokens_list = list()

    for word in words:
        try:
            tokens_list.append( tokenizer.word_index[ word ] )
        except:
            tokens_list.append(tokenizer.word_index['i dont know'])

    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

enc_model , dec_model = make_inference_models()

def chat_bot_lstm(cau_hoi):
    states_values = enc_model.predict( str_to_tokens( cau_hoi ) )
#     print(states_values)
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]
    return ( decoded_translation[1:-4])

In [16]:
print(chat_bot_lstm('hello'))

1/1 [==============================] - 1s 1s/step



In [44]:
def prediction(user_input_padded,thershold=0.002):
    predict = model_1.predict(user_input_padded)
    print(user_input_padded)
    print(predict)
    if predict[0] >= thershold:
        return 'Information Tag'
    else:
        return 'Non-Information Tag'

In [18]:
def remove_padding(input_padded):
    unpadded_input = []
    for sequence in input_padded:
        sentence = [word for word in sequence if word != 0]  # Assuming 0 is the padding token
        # print(sentence)
        unpadded_input.append(sentence)
    return unpadded_input

# Reverse the tokenization
def reverse_tokenization(sequences, tokenizer):
    original_sentences = []
    for sequence in sequences:
        # print(sequence)
        original_sentence = tokenizer.sequences_to_texts([sequence])[0]
        # print(original_sentence)
        original_sentences.append(original_sentence)
    return original_sentences

In [19]:
def pos_tokenize(text):
  words = word_tokenize(text)
  tags = pos_tag(words)
  selected_words = [word for word, tag in tags if tag in ['NN', 'NNP','CD','JJ']]
  return selected_words

In [68]:
def classification_tags(predict):
  if predict == 'Information Tag':
      original_user_input = temp
      price_tags = [
    'cost', 'price', 'how much', 'sell', 'budget', 'quotation', 'amount',"lowest price","total"
    'market value', 'pricing information', 'price list', 'price range',"final cost","sale price"
    'purchase cost', 'price point', 'expense', 'valuation',"rate", "cost range","price estimation"
    'invoice','list price', 'top price', 'bottom price',"retail price","expenditure"
    'final price', 'bill',"sold","purchase","spend","pay","price tag","current price",'much'
]
      screen_resolution_tags = [
    'screen resolution', 'clarity', 'Full HD', 'Retina display', 'contrast level',
    'color reproduction', 'OLED display', 'LCD display', 'brightness setting',
    'glare-resistant screen', 'refresh rate', '4K resolution',
    'QHD display', 'IPS technology', 'AMOLED screen', 'pixel density',
    'display quality', 'picture quality', 'PPI', 'HD', 'visual sharpness',
    'display specs', 'image definition', 'aspect ratio', 'contrast ratio',
    'color accuracy', 'brightness levels', 'display type', 'screen clarity',
    'viewing experience', 'pixels','screen','size','screen size','resolution'
]
      phone_memory_keywords = [
    "memory", "storage", "GB", "TB", "storage space", "capacity", "ram",
    "internal memory", "Built-in Storage", "Phone Storage", "External Storage",
    "expandable storage", "Memory Card Slot", "microSD Support", "microSD Slot",
    "UFS", "Memory Upgrade", "Storage Expansion", "rom", "system memory",
    "user available memory", "cache memory", "phone memory", "device storage",
    "onboard storage", "Secure Storage", "Memory Card Capacity",
    "Dual SIM with Memory Card Slot", "LPDDR4", "LPDDR5",
    "Read-Only Memory", "Flash Memory", "eMMC", "NVMe",
    "USB OTG Support", "Memory Efficiency", "High-Speed Memory",
    "Memory Bandwidth", "Storage Speed", "Memory Card Compatibility"
]

      # information_tags = ['information']
      tokenize = pos_tokenize(original_user_input)
      # print(tokenize)
      for keyword in tokenize:
        if keyword in price_tags:
          filtered_elements = [element for element in tokenize if element not in price_tags]
          concatenated_string = ' '.join(filtered_elements)
          return 'price_tag', concatenated_string
        elif keyword in screen_resolution_tags:
          filtered_elements = [element for element in tokenize if element not in screen_resolution_tags]
          concatenated_string = ' '.join(filtered_elements)
          return 'resolution_tag', concatenated_string
        elif keyword in phone_memory_keywords:
          filtered_elements = [element for element in tokenize if element not in screen_resolution_tags]
          concatenated_string = ' '.join(filtered_elements)
          return 'memory_tag', concatenated_string
  else:
    original_user_input = temp
    return "Non-Information Tag",chat_bot_lstm(original_user_input)


In [86]:
user_input = input("Enter your question: ")
temp = user_input
# original_user_input = user_input
# Preprocess and tokenize user input
user_input = preprocess_and_tokenize_sentences(user_input)
user_input_sequences = tokenizer.texts_to_sequences(user_input)
user_input_padded = pad_sequences(user_input_sequences, maxlen=max_len, padding='post')
user_input_padded

Enter your question: what is the storage of Iphone 15?


array([[ 15,   9,   2, 372,  13,  45,  98,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0]], dtype=int32)

In [87]:
predict = prediction(user_input_padded)
predict

1/1 [==============================] - 0s 18ms/step
[[ 15   9   2 372  13  45  98   0   0   0   0   0   0   0   0   0   0   0]]
[[2.8725024e-05]]


'Non-Information Tag'

In [88]:
classification_tags(predict)

1/1 [==============================] - 0s 26ms/step


('Non-Information Tag',
 'yes we can a payment of the payment of the payment of the payment')